<a href="https://colab.research.google.com/github/Ekansh-Sethia/vehicle_detection_CV_YOLO/blob/main/Vehicle_Detection_YOLO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ultralytics -q

import cv2
import requests
import numpy as np
from ultralytics import YOLO
from google.colab.patches import cv2_imshow

# 1. Load the newest YOLO model (Nano version for extreme speed)
# This is cached automatically in your Colab session
model = YOLO('yolo11n.pt')

def run_detection_on_image_url(image_url):
    # Fetch image
    try:
        resp = requests.get(image_url, stream=True, timeout=10).raw
        image = np.asarray(bytearray(resp.read()), dtype="uint8")
        frame = cv2.imdecode(image, cv2.IMREAD_COLOR)
        if frame is None: return
    except Exception as e:
        print(f"Error: {e}")
        return

    # 3. Optimized Inference
    # vehicle_classes: 2 (car), 3 (motorcycle), 5 (bus), 7 (truck)
    results = model.predict(
        source=frame,
        classes=[2, 3, 5, 7],
        conf=0.25,      # Adjusted threshold for better recall
        save=False,
        verbose=False
    )

    # 4. Result Processing
    # Use the built-in .plot() method which is written in C++ (much faster than Python loops)
    annotated_frame = results[0].plot(line_width=1, font_size=0.5)

    # Add a clean counter
    count = len(results[0].boxes)
    cv2.putText(annotated_frame, f'Vehicles: {count}', (20, 40),
                cv2.FONT_HERSHEY_DUPLEX, 0.6, (0, 0, 255), 1)

    # 5. Display
    cv2_imshow(annotated_frame)


# def run_tensorflow_detection_webcam():
#     """
#     This function runs a live vehicle detection loop using a webcam and a
#     pre-trained SSD MobileNet V2 model from TensorFlow Hub.
#     """
#     # --- Model Loading ---
#     # Load a pre-trained object detection model from TensorFlow Hub
#     model_url = "https://tfhub.dev/tensorflow/ssd_mobilenet_v2/2"
#     try:
#         print("Loading TensorFlow model...")
#         model = hub.load(model_url)
#         print("TensorFlow model loaded successfully.")
#     except Exception as e:
#         print(f"Error loading model from TensorFlow Hub: {e}")
#         print("Please ensure you have an internet connection and have run 'pip install tensorflow tensorflow_hub'.")
#         return

#     # COCO dataset labels (the model was trained on this)
#     # We need to find the indices for vehicles.
#     # From the COCO label map: 3='car', 4='motorcycle', 6='bus', 8='truck'
#     vehicle_class_ids = [3, 4, 6, 8]

#     # A simple dictionary to map class IDs to names for display
#     class_names = {3: 'car', 4: 'motorcycle', 6: 'bus', 8: 'truck'}

#     # Initialize video capture from the default webcam (index 0)
#     cap = cv2.VideoCapture(0)

#     if not cap.isOpened():
#         print("Error: Could not open webcam.")
#         return

#     print("Starting live detection with TensorFlow... Press 'q' to quit.")

#     while True:
#         # Read a frame from the webcam
#         ret, frame = cap.read()
#         if not ret:
#             print("Error: Could not read frame from webcam.")
#             break

#         # Get frame dimensions
#         height, width, _ = frame.shape

#         # --- TensorFlow Inference ---
#         # Convert the frame (BGR) to RGB and then to a TensorFlow tensor
#         gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
#         input_tensor = tf.convert_to_tensor(gray_frame)

#         # The model expects a batch of images, so we add an axis.
#         input_tensor = input_tensor[tf.newaxis, ...]

#         # Run inference
#         results = model(input_tensor)

#         # The results are tensors, so we convert them to numpy arrays.
#         # We also remove the batch dimension using [0].
#         scores = results['detection_scores'][0].numpy()
#         classes = results['detection_classes'][0].numpy().astype(np.int32)
#         boxes = results['detection_boxes'][0].numpy()

#         vehicle_count = 0
#         # Iterate through the detections
#         for i in range(len(scores)):
#             # Apply a confidence threshold
#             if scores[i] > 0.2:
#                 # Check if the detected object is a vehicle
#                 if classes[i] in vehicle_class_ids:
#                     vehicle_count += 1

#                     # --- Bounding Box Calculation ---
#                     # The box coordinates are normalized (0 to 1), so we scale them
#                     ymin, xmin, ymax, xmax = boxes[i]
#                     x1 = int(xmin * width)
#                     y1 = int(ymin * height)
#                     x2 = int(xmax * width)
#                     y2 = int(ymax * height)

#                     # Get class name and confidence score
#                     class_name = class_names.get(classes[i], 'N/A')
#                     confidence = scores[i]

#                     #Draw Bounding Box with Label
#                     cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
#                     label = f'{class_name} {confidence:.2f}'
#                     cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

#         # Display Information
#         cv2.putText(frame, f'Detected Vehicles: {vehicle_count}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)

#         # resulting frame
#         cv2_imshow(frame)

#         # Check for the 'q' key to exit the loop
#         if cv2.waitKey(1) & 0xFF == ord('q'):
#             break


#     cap.release()
#     cv2.destroyAllWindows()
#     print("Detection stopped.")


if __name__ == "__main__":

    example_image_url = "https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTK0j4R95WiLCP5HonXA2ttC-Ocybay2exX7w&s"
    run_detection_on_image_url(example_image_url)



